In [ ]:
# VGQEC for 5 qubits

import pennylane as qml
import numpy as np

dev_sample = qml.device("default.mixed", wires=11, shots=1)
dev_analytic = qml.device("default.mixed", wires=11, shots=1000)

def encoding_variation_circuit_5(wires, alpha):
    for wire in wires:
        qml.RZ(alpha[0], wires=wire)

    qml.IsingZZ(alpha[1], wires=[0, 1])
    qml.IsingZZ(alpha[1], wires=[3, 4])

    qml.RX(alpha[2], wires=1)
    qml.RX(alpha[2], wires=3)

    qml.IsingZZ(alpha[1], wires=[1, 2])
    qml.IsingZZ(alpha[1], wires=[2, 3])

    qml.RX(alpha[2], wires=2)

    qml.IsingZZ(alpha[1], wires=[1, 2])
    qml.IsingZZ(alpha[1], wires=[2, 3])

    qml.RX(alpha[2], wires=1)
    qml.RX(alpha[2], wires=3)

    qml.IsingZZ(alpha[1], wires=[0, 1])
    qml.IsingZZ(alpha[1], wires=[1, 2])
    qml.IsingZZ(alpha[1], wires=[2, 3])
    qml.IsingZZ(alpha[1], wires=[3, 4])

    qml.RX(alpha[2], wires=0)
    qml.RX(alpha[2], wires=2)
    qml.RX(alpha[2], wires=4)

    qml.IsingZZ(alpha[1], wires=[1, 2])
    qml.IsingZZ(alpha[1], wires=[2, 3])

    qml.RX(alpha[2], wires=1)
    qml.RX(alpha[2], wires=2)
    qml.RX(alpha[2], wires=3)

    qml.IsingZZ(alpha[1], wires=[1, 2])
    qml.IsingZZ(alpha[1], wires=[2, 3])

    qml.RX(alpha[2], wires=2)

    qml.IsingZZ(alpha[1], wires=[1, 2])
    qml.IsingZZ(alpha[1], wires=[2, 3])

    qml.RX(alpha[2], wires=1)
    qml.RX(alpha[2], wires=3)

    qml.IsingZZ(alpha[1], wires=[0, 1])
    qml.IsingZZ(alpha[1], wires=[3, 4])

    qml.RX(alpha[2], wires=1)
    qml.RX(alpha[2], wires=3)

    qml.IsingZZ(alpha[1], wires=[1, 2])
    qml.IsingZZ(alpha[1], wires=[2, 3])  

    qml.RX(alpha[2], wires=2)

    qml.IsingZZ(alpha[1], wires=[1, 2])
    qml.IsingZZ(alpha[1], wires=[2, 3])  

    qml.RX(alpha[2], wires=1)
    qml.RX(alpha[2], wires=3)

    qml.IsingZZ(alpha[1], wires=[0, 1])
    qml.IsingZZ(alpha[1], wires=[3, 4])
    
    qml.RX(alpha[2], wires=0)
    qml.RX(alpha[2], wires=4)

    for wire in wires:
        qml.RZ(alpha[0], wires=wire)

        
def encoding_circuit_513(wires):
    for wire in wires[1:]:
        qml.CNOT(wires=[0, wire])
    for wire in wires:
        qml.Hadamard(wires=wire)
    
    qml.IsingZZ(-np.pi/2, wires=[4, 0])
    qml.IsingZZ(-np.pi/2, wires=[0, 1])
    qml.IsingZZ(-np.pi/2, wires=[1, 2])
    qml.IsingZZ(-np.pi/2, wires=[2, 3])
    qml.IsingZZ(-np.pi/2, wires=[3, 4])

    
def apply_noise_layer(gamma):

#     for wire in range(5):
#         qml.AmplitudeDamping(gamma, wires=wire)
    
    qml.AmplitudeDamping(1.0, wires=0)

    
# --- SYNDROME MEASUREMENT ---
def measure_syndrome():
    
    # S1 = X Z Z X I
    qml.Hadamard(7)
    qml.Hadamard(0); qml.Hadamard(3)
    for i in [0, 1, 2, 3]:
        qml.CNOT(wires=[i, 7])
    qml.Hadamard(0); qml.Hadamard(3)

    # S2 = I X Z Z X
    qml.Hadamard(8)
    qml.Hadamard(1); qml.Hadamard(4)
    for i in [1, 2, 3, 4]:
        qml.CNOT(wires=[i, 8])
    qml.Hadamard(1); qml.Hadamard(4)

    # S3 = X I X Z Z
    qml.Hadamard(9)
    qml.Hadamard(0); qml.Hadamard(2)
    for i in [0, 2, 3, 4]:
        qml.CNOT(wires=[i, 9])
    qml.Hadamard(0); qml.Hadamard(2)

    # S4 = Z X I X Z
    qml.Hadamard(10)
    qml.Hadamard(1); qml.Hadamard(3)
    for i in [0, 1, 3, 4]:
        qml.CNOT(wires=[i, 10])
    qml.Hadamard(1); qml.Hadamard(3)
    
# --- CORRECTION MAPPING ---
def decode_and_correct(syndrome):
    syndrome_map = {
        (0, 0, 0, 0): None,
        (0, 0, 0, 1): (0, 'X'), (1, 0, 0, 0): (1, 'X'), 
        (1, 1, 0, 0): (2, 'X'), (0, 1, 1, 0): (3, 'X'), 
        (0, 0, 1, 1): (4, 'X'), (1, 0, 1, 0): (0, 'Z'), 
        (0, 1, 0, 1): (1, 'Z'), (0, 0, 1, 0): (2, 'Z'), 
        (1, 0, 0, 1): (3, 'Z'), (0, 1, 1, 1): (4, 'Z'),
        (1, 0, 1, 1): (0, 'Y'), (1, 1, 0, 1): (1, 'Y'), 
        (1, 1, 1, 0): (2, 'Y'), (1, 1, 1, 1): (3, 'Y'), 
        (0, 1, 0, 0): (4, 'Y')
    }
    key = tuple(syndrome)
    if key not in syndrome_map or syndrome_map[key] is None:
        return []
    qubit, op = syndrome_map[key]
    return [getattr(qml, f'Pauli{op}')(wires=qubit)]


def decode_variational_5(beta, wires, L = 3):
    rzz_pairs = [
        (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6),
        (1, 2), (1, 3), (1, 4), (1, 5), (1, 6),
        (2, 3), (2, 4), (2, 5), (2, 6),
        (3, 4), (3, 5), (3, 6),
        (4, 5), (4, 6),
        (5, 6)
    ]

    for wire in wires:
        qml.RZ(beta[1], wires=wire)

    for l in range(L):
        for wire in wires:
            qml.RX(beta[0], wires=wire)
            qml.RZ(beta[1], wires=wire)        

        for (i, j) in rzz_pairs:
            qml.IsingZZ(beta[2], wires=[wires[i], wires[j]])

    for wire in wires:
        qml.RX(beta[0], wires=wire)
        qml.RZ(beta[1], wires=wire) 
        

        
@qml.qnode(dev_sample)
def apply_noise_and_measure(gamma, alpa):
    encoding_circuit_513(dev_sample.wires)
    
    encoding_variation_circuit_5(dev_sample.wires, alpha)
    
    apply_noise_layer(gamma)
    
    decode_variational_5(beta, dev_sample.wires)
    
    measure_syndrome()
    
    return qml.sample(wires=[7, 8, 9, 10])


@qml.qnode(dev_sample)
def apply_correction(correction_ops=[]):
    for op in correction_ops:
        op.queue()
        
    return qml.density_matrix(wires=[0])  # logical qubit


if __name__ == '__main__':
    
    gamma = 0.0
    
    alpha = [np.pi/2, np.pi/2, np.pi/2]
    beta = [np.pi/2, np.pi/2, np.pi/2]

    print(qml.draw_mpl(apply_noise_and_measure(gamma, alpha)))
    
    all_fc = []
    all_gamma = []
    
    iterations = 10 
    
    while(gamma<=0.5):
        
        totoal_fid = 0
        
        for i in range(iterations):
            syndrome = apply_noise_and_measure(gamma, alpha)
            
            correction_ops = decode_and_correct(list(syndrome))
            
            result = apply_correction(correction_ops)

            ent_fidelity = np.mean(result == 0)  # Measure probability of |0>
            totoal_fid += ent_fidelity

            print("Syndrome:", list(syndrome))
            print("Correction ops:", correction_ops)

            print("Fidelity after recovery:", ent_fidelity)
            print("-" * 40)
        
        all_fc.append(compute_fc(totoal_fid/iterations)) # Take avg of all iterations
        all_gamma.append(gamma)
        
        print("gamma = ", gamma, "; avg fid = ", compute_fc(totoal_fid/iterations), "\n")
        
        gamma+=0.05
        
        
        
xpoints = all_gamma
ypoints = all_fc

plt.title("Channel Fidelity vs Gamma")
plt.plot(xpoints, ypoints)
plt.show()